$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

$\textbf{Preliminaries}$ 

The libraries used here are:

In [1]:
import os
import numpy as np
from itertools import product
from multiprocessing import Pool

In [2]:
MG5_Path = os.path.relpath('/Collider/MG5_aMC_v3_1_0/bin/mg5_aMC', os.getcwd())
Model_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), '00_FeynRules_UFO','Mod2_VLQ_UFO')
Paramcards_folder_path = os.path.join(os.path.dirname(os.getcwd()), 'Param_Cards')

In [3]:
Folder = 'Data_Parton_Level'

Betards = ['woRHC']
zp_limits = ['zp_upper_limit', 'zp_lower_limit']

In [4]:
generation_options = {'ta_ta_virtual_contribution':  'generate p p > ta ta QED = 0 $$ zp', 
                      'ta_ta_real_contribution': 'generate p p > zp > ta ta QED = 0',
                      'ta_ta_total_contribution': 'generate p p > ta ta QED = 0'}

In [5]:
M_Us = np.arange(2000,3500,500)
g_Us = np.arange(0.5,1.5,0.5)
g_Zps = np.arange(0.5,2,0.5)

# M_Us = [1500]
# g_Us = [1.0]
# g_Zps = [0.5]

In [6]:
# t = 0
# for mu, gu, gzp, Betard, zp_limits in product(M_Us, g_Us, g_Zps, Betards, zp_limits): #It takes 5 minutes by combination
#     t = t + 5
# print(t)

In [7]:
seeds = []
def semilla():
    seed = np.random.randint(1,5000)
    if (seed in seeds):
        return semilla()
    else: seeds.append(seed)
    return seed

def generate_data(process):
    global label, Betard, zp_limit, Folder
    
    nruns = 6
    
    param_card = os.path.join(Paramcards_folder_path, Betard, zp_limit, f'param_card_mu_{mu}_gu{gu}_gzp_{gzp}.dat')
    if not os.path.exists(param_card):
        raise Exception(f"Param card {param_card} not generated")
            
    script_lines = [f'import model {Model_path}', 'define ta = ta+ ta-', 'define lq = vlq vlq~']    
    script_lines.append(generation_options[process])
    script_lines.append(f'output {os.path.join(os.getcwd(), Folder, Betard, zp_limit, process, label)} -nojpeg')
    
    for i in range(nruns):
        script_lines.append(f'launch {os.path.join(os.getcwd(), Folder, Betard, zp_limit, process, label)} -m')
        script_lines.append('8')
        script_lines.append('done')
        script_lines.append(f'set iseed {semilla()}')
        script_lines += ['set nevents 50000', 'set cut_decays True', 'set ptb 30', 'set ptj 20', 'set ptl 20', 'set etab 2.5', 'set pt_min_pdg {15: 30}', 'set eta_max_pdg {15: 2.5}']
        script_lines.append('set mxx_min_pdg {15:100}')
        script_lines.append('set sde_strategy 1')
        script_lines.append(param_card)

    script = open (f'script_{Betard}_{zp_limit}_{label}_{process}.txt','w')
    for line in script_lines: script.write(line + '\n')
    script.close()
    !{MG5_Path} script_{Betard}_{zp_limit}_{label}_{process}.txt

In [ ]:
for mu, gu, gzp, Betard, zp_limit in product(M_Us, g_Us, g_Zps, Betards, zp_limits): #It takes - minutes by combination
    
    label = f'mu_{mu}_gu_{gu}_gzp_{gzp}'
    with Pool(3) as p: p.map(generate_data, generation_options.keys())

************************************************************
*                                                          *
*                     W E L C O M E to                     *
*              M A D G R A P H 5 _ a M C @ N L O           *
*                                                          *
*                                                          *
*                 *                       *                *
*                   *        * *        *                  *
*                     * * * * 5 * * * *                    *
*                   *        * *        *                  *
*                 *                       *                *
*                                                          *
*         VERSION 3.1.0                 2021-03-30         *
*                                                          *
*    The MadGraph5_aMC@NLO Development Team - Find us at   *
*    https://server06.fynu.ucl.ac.be/projects/madgraph     *
*                       

In [ ]:
!rm script*

In [ ]:
!touch acabo_Generate_Data.txt